In [3]:
# 数据预处理和特征工程
import sys
import os
from pathlib import Path
# 解释：将当前目录的父目录加入到sys.path中，这样就可以在当前目录下导入父目录的模块
parent = Path(os.path.abspath("")).resolve().parents[0]
if parent not in sys.path:
    sys.path.insert(0, str(parent))


In [4]:
# 导入必要的库
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split

# 数据加载
data_path = '/home/yanglin/Federated-Time-Series-Forecasting/dataset/mergeData.csv'  # 请替换为你的数据文件路径
data = pd.read_csv(data_path)
data.columns = data.columns.str.strip()

print(data.columns)


Index(['vehicle_id', 'position_x', 'position_y', 'position_z', 'heading',
       'speed', 'speed_command', 'mileage_distance', 'power_on_time', 'soc',
       'fuel_level', 'local_time', 'chassis_mode', 'estop', 'charge_status',
       'task_state_running', 'task_state_estop', 'task_state_lock',
       'task_stage', 'current_task', 'error_code', 'target_location',
       'vesselVisitID', 'mission_type', 'container1_type', 'container2_type',
       'vehicle_mode', 'missionID', 'port'],
      dtype='object')


## 数据探索

In [5]:
print(data.head())


  vehicle_id  position_x  position_y  position_z   heading  speed  \
0       A022  262.064362  142.446716           0 -0.000112    0.0   
1       A022  262.065826  142.444717           0 -0.000160    0.0   
2       A022  262.068390  142.442184           0 -0.000126    0.0   
3       A022  262.067108  142.441650           0 -0.000238    0.0   
4       A022  262.066406  142.442871           0 -0.000224    0.0   

   speed_command  mileage_distance  power_on_time  soc  ...  current_task  \
0            0.0             11330       30912088   83  ...          DSCH   
1            0.0             11330       30912090   83  ...          DSCH   
2            0.0             11330       30912092   83  ...          DSCH   
3            0.0             11330       30912096   83  ...          DSCH   
4            0.0             11330       30912098   83  ...          DSCH   

  error_code        target_location vesselVisitID  mission_type  \
0                QCTP.Q306.106/QCTP       HED_07B      

In [22]:
print(data.describe())


         position_x    position_y  position_z       heading         speed  \
count  11754.000000  11754.000000     11754.0  11754.000000  11754.000000   
mean     -18.045472    -15.135898         0.0      0.612750      0.288582   
std      210.982803    162.362195         0.0      1.024565      1.018605   
min     -512.456360   -218.794800         0.0     -3.141563     -0.619502   
25%      -89.142870   -218.725494         0.0      0.000351      0.000000   
50%      -13.019513     80.675076         0.0      0.000921      0.000000   
75%      154.221664    122.477875         0.0      1.538339      0.000000   
max      326.494904    211.914383         0.0      3.141550      5.741405   

       speed_command  mileage_distance  power_on_time           soc  \
count   11754.000000      11754.000000   1.175400e+04  11754.000000   
mean        0.297355      11450.865152   3.093014e+07     75.413987   
std         1.053031        351.698143   4.493682e+04      7.556484   
min        -0.300000  

In [23]:
print(data.isnull().sum())

vehicle_id            0
position_x            0
position_y            0
position_z            0
heading               0
speed                 0
speed_command         0
mileage_distance      0
power_on_time         0
soc                   0
fuel_level            0
local_time            0
chassis_mode          0
estop                 0
charge_status         0
task_state_running    0
task_state_estop      0
task_state_lock       0
task_stage            0
current_task          0
error_code            0
target_location       0
vesselVisitID         0
mission_type          0
container1_type       0
container2_type       0
vehicle_mode          0
missionID             0
port                  0
dtype: int64


##   数据清洗

In [6]:
# 数据清洗
# 分别处理数值列和非数值列
for column in data.columns:
    # 如果该列中不存在缺失值，跳过该列
    if data[column].isnull().sum() == 0:
        continue
    if pd.api.types.is_numeric_dtype(data[column]):
        # 用中位数填充数值列
        data[column].fillna(data[column].median(), inplace=True)
    else:
        # 用众数填充非数值列
        data[column].fillna(data[column].mode()[0], inplace=True)

In [7]:
# 检查每一列是否包含非数值型的字符串
for column in data.columns:
    if data[column].dtype == 'object' and not data[column].str.isnumeric().all():
        non_numeric_values = data[column].loc[~data[column].str.isnumeric()]
        print(f"Column '{column}' contains non-numeric values:")
        print(non_numeric_values)


Column 'vehicle_id' contains non-numeric values:
0        A022
1        A022
2        A022
3        A022
4        A022
         ... 
11749    A028
11750    A028
11751    A028
11752    A028
11753    A028
Name: vehicle_id, Length: 11754, dtype: object
Column 'local_time' contains non-numeric values:
0           2023-01-28 08:10:02
1           2023-01-28 08:10:05
2           2023-01-28 08:10:08
3           2023-01-28 08:10:11
4           2023-01-28 08:10:14
                  ...          
11749       2023-01-29 22:03:43
11750       2023-01-29 22:03:46
11751       2023-01-29 22:03:49
11752       2023-01-29 22:03:52
11753       2023-01-29 22:03:55
Name: local_time, Length: 11754, dtype: object
Column 'estop' contains non-numeric values:
0           False
1           False
2           False
3           False
4           False
           ...   
11749       False
11750       False
11751       False
11752       False
11753       False
Name: estop, Length: 11754, dtype: object
Column 'task_state

In [8]:
# 数据清洗：处理数据前后存在空格问题
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = data[column].str.strip()

In [9]:
# 转换布尔值
for column in data.columns:
    if data[column].dtype == 'object' and data[column].str.contains('True|False', na=False).any():
        data[column] = data[column].str.strip().map({'True': 1, 'False': 0})

In [10]:
# 处理或删除异常值，示例：删除速度异常的记录
data = data[data['speed'] >= 0]  # 假设速度不可能小于0

In [8]:
# 类别型列进行独热编码
categorical_cols = ['task_stage', 'current_task', 'error_code', 'target_location', 'vesselVisitID', 
                    'mission_type', 'container1_type', 'container2_type', 'vehicle_mode']

# 检查列是否存在
categorical_cols = [col for col in categorical_cols if col in data.columns]

data = pd.get_dummies(data, columns=categorical_cols)
print(data.head())

  vehicle_id  position_x  position_y  position_z   heading  speed  \
0       A032 -584.685669  142.291031         0.0 -0.002938    0.0   
1       A032 -584.686157  142.295425         0.0 -0.002805    0.0   
2       A032 -584.689758  142.296387         0.0 -0.002790    0.0   
3       A032 -584.688843  142.298386         0.0 -0.002682    0.0   
4       A032 -584.688232  142.299530         0.0 -0.002690    0.0   

   speed_command  mileage_distance  power_on_time  soc  ...  \
0            0.0           10179.0     33094818.0   53  ...   
1            0.0           10179.0     33094820.0   53  ...   
2            0.0           10179.0     33094824.0   53  ...   
3            0.0           10179.0     33094826.0   53  ...   
4            0.0           10179.0     33094830.0   53  ...   

   mission_type_   RECEIVE container1_type_0  container1_type_20  \
0                    False              True               False   
1                    False              True               False   
2 

In [11]:
#用目标编码替换独热编码
from category_encoders import TargetEncoder

# 定义需要目标编码的类别型特征
categorical_cols = ['task_stage', 'current_task', 'error_code', 'target_location', 'vesselVisitID', 
                    'mission_type', 'container1_type', 'container2_type', 'vehicle_mode']

# 检查列是否存在
categorical_cols = [col for col in categorical_cols if col in data.columns]
data['needs_charging'] = data['charge_status'].apply(lambda x: 1 if x in [1, 2, 3, 5] else 0)

# 使用目标编码替代独热编码
target_encoder = TargetEncoder(cols=categorical_cols)

# 注意：fit_transform方法需要传入标签数据
data = target_encoder.fit_transform(data, data['needs_charging'])



## 特征提取

In [12]:
# 时间特征提取
data['local_time'] = pd.to_datetime(data['local_time'])
data['hour'] = data['local_time'].dt.hour
data['day_of_week'] = data['local_time'].dt.dayofweek


# 滞后特征，示例：计算前5分钟的平均速度
data.sort_values('local_time', inplace=True)
data['avg_speed_5min'] = data['speed'].rolling(window=5).mean()


# 标签转换，将charge_status处理为二分类目标
# data['needs_charging'] = data['charge_status'].apply(lambda x: 1 if x in [1, 3] else 0)

# 数据切分
features = data.drop(columns=['charge_status', 'needs_charging', 'local_time'])
labels = data['needs_charging']


X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

X_train.head()


,vehicle_id,position_x,position_y,position_z,heading,speed,speed_command,mileage_distance,power_on_time,soc,...,vesselVisitID,mission_type,container1_type,container2_type,vehicle_mode,missionID,port,hour,day_of_week,avg_speed_5min
5797,A022,154.233978,-218.729309,0,0.000933,0.000000,0.000000,11339,30930888,72,...,0.0,0.030909,0.018724,0.015729,0.0,365fbfb4-9ecc-11ed-ac70-0242ac110030,tianhaiheda,13,5,0.000000
7845,A022,154.221878,-218.728104,0,0.000567,0.000000,0.000000,11339,30936910,72,...,0.0,0.030909,0.018724,0.015729,0.0,365fbfb4-9ecc-11ed-ac70-0242ac110030,tianhaiheda,15,5,0.000000
11646,A028,-173.617371,13.083996,0,-0.000052,3.529296,3.234201,11419,31048816,59,...,0.0,0.000000,0.000000,0.015729,0.0,6c9eb5ba-9fdc-11ed-8e85-0242ac11003c,changjinxiantai,21,6,5.109102
719,A022,-269.873993,-97.462982,0,-0.004478,1.236010,1.225601,11332,30914202,79,...,0.0,0.000000,0.000000,0.015729,0.0,4d15657e-9ea4-11ed-8856-0242ac110030,tianhaiheda,8,5,0.962830
1087,A022,-248.316193,2.064377,0,-0.003248,0.000000,0.000000,11334,30915288,78,...,0.0,0.000000,0.000000,0.015729,0.0,af0ad1c2-9ea6-11ed-8e1d-0242ac110030,tianhaiheda,9,5,0.000000


In [10]:
# 检查训练集和测试集的大小
print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)

Training set size: (9358, 30)
Testing set size: (2340, 30)


In [11]:
# 将特征和标签合并
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 9358 entries, 5797 to 7319
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   vehicle_id          9358 non-null   object 
 1   position_x          9358 non-null   float64
 2   position_y          9358 non-null   float64
 3   position_z          9358 non-null   int64  
 4   heading             9358 non-null   float64
 5   speed               9358 non-null   float64
 6   speed_command       9358 non-null   float64
 7   mileage_distance    9358 non-null   int64  
 8   power_on_time       9358 non-null   int64  
 9   soc                 9358 non-null   int64  
 10  fuel_level          9358 non-null   int64  
 11  chassis_mode        9358 non-null   int64  
 12  estop               9358 non-null   int64  
 13  task_state_running  9358 non-null   int64  
 14  task_state_estop    9358 non-null   int64  
 15  task_state_lock     9358 non-null   int64  
 16  task_sta

## 保存文件

In [12]:
# 保存合并后的数据集为CSV文件
train_data.to_csv('/home/yanglin/Federated-Time-Series-Forecasting/dataset/yl/train_data.csv', index=False)
test_data.to_csv('/home/yanglin/Federated-Time-Series-Forecasting/dataset/yl/test_data.csv', index=False)